## Imports

In [2]:
import os
import numpy as np
from PIL import Image
import time
import csv
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


In [3]:
cuda = torch.cuda.is_available()
cuda

True

In [58]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        #print("HI")
        #print(self.file_list[index])
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        if(self.file_list[index] == "test_verification/299317.jpg"):
            print("Image" , img)
            print("Shape",img.shape)
        label = self.target_list[index]
        return img, label

#### Parse the given directory to accumulate all the images

In [28]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                temp = filei.split('/')[-1]
                #ID_list.append(root.split('/')[-1])
                ID_list.append(int(temp.split('.')[0]))
    print(img_list[:10])
    print(ID_list[:10])
    # construct a dictionary, where key and value correspond to ID and target
    uniqueID_list = list(set(ID_list))
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    label_list = [target_dict[ID_key] for ID_key in ID_list]

    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, ID_list, class_n , target_dict

In [29]:
img_list, label_list, class_n ,dictionary = parse_data('test_verification')
#print(dictionary)

['test_verification/317667.jpg', 'test_verification/234228.jpg', 'test_verification/351500.jpg', 'test_verification/207801.jpg', 'test_verification/349870.jpg', 'test_verification/325224.jpg', 'test_verification/331654.jpg', 'test_verification/230454.jpg', 'test_verification/316397.jpg', 'test_verification/311255.jpg']
[317667, 234228, 351500, 207801, 349870, 325224, 331654, 230454, 316397, 311255]
#Images		#Labels
169392		169392


In [30]:
class ConvBNRelU(nn.Module):
    def __init__(self,in_channel, out_channel, kernel_size, stride):
        super(ConvBNRelU, self).__init__()
        if(kernel_size==1):
            self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, stride=stride, bias=False)
        else:
            if(in_channel == out_channel):
                self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, stride=stride, padding=1, groups = out_channel,bias=False)
            else:
                self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, stride=stride, padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu6 = nn.ReLU6(inplace=True)
    
    def forward(self,x):
        return self.relu6(self.bn1(self.conv1(x)))
        
#ConvBNRelU(16,16,6,2)  

In [31]:
class InvertedResidual(nn.Module):
    
    def __init__(self,in_channel, out_channel,expansion_factor,stride):
        super(InvertedResidual, self).__init__()
        self.stride=stride
        self.expansion_factor = expansion_factor
        
        if(self.expansion_factor == 1):
            self.convbn3 = ConvBNRelU(in_channel, in_channel, 3, stride)
        else:
            self.convbn1 = ConvBNRelU(in_channel, expansion_factor*in_channel, 1, 1)
            self.convbn3 = ConvBNRelU(expansion_factor*in_channel, expansion_factor*in_channel, 3, stride)
        
        self.conv = nn.Conv2d(expansion_factor*in_channel, out_channel, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        
        if(self.stride==1):
            self.shortcut = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride, bias=False)
            
    
    def forward(self,x):
        if(self.stride==1):
            if(self.expansion_factor==1):
                output= self.bn1(self.conv(self.convbn3(x)))
            else:
                output=self.bn1(self.conv(self.convbn3(self.convbn1(x))))
            return output + self.shortcut(x)
            return output
        else:
            return self.bn1(self.conv(self.convbn3(self.convbn1(x))))
            
        
        
        
# net=nn.Sequential(*[InvertedResidual(32,16,1,1)])   
# print(net)        

## CNN Model with Residual Block 

In [70]:
class Network_MobileNet(nn.Module):
    def __init__(self, input_channels, hidden_sizes, num_classes, feat_dim, expansion_factor_list, strides_list, repeat_list):
        super(Network_MobileNet, self).__init__()
        self.layers = []
        
        self.layers.append(ConvBNRelU(input_channels,hidden_sizes[0], 3, 2))
        #self.layers.append(nn.BatchNorm2d(hidden_sizes[0]))
        #self.layers.append(nn.ReLU6(inplace=True))
        
        for idx in range(len(hidden_sizes)-1):
            
            for j in range(repeat_list[idx]):
                
                if(j==0):
                    self.layers.append(InvertedResidual(hidden_sizes[idx], hidden_sizes[idx+1], expansion_factor_list[idx],strides_list[idx]))
                else:
                    self.layers.append(InvertedResidual(hidden_sizes[idx+1], hidden_sizes[idx+1], expansion_factor_list[idx],1))
                    
        
        self.layers.append(ConvBNRelU(hidden_sizes[-1],1280, 1, 1))
        self.layers = nn.Sequential(*self.layers)
        self.classifier = nn.Sequential(*[nn.Linear(1280,2300,bias=True)])
        #print(self.layers[-1].conv1.weight.shape)
        #self.classifier = nn.Linear(1280,2300,bias=True)
        #self.classifier = nn.Linear(1280,2300,bias=True)
        
        #print(self.layers)
        
        #print(self.layers)
        #self.linear_label = nn.Linear(self.hidden_sizes[-2], self.hidden_sizes[-1], bias=False)
        

    
    def forward(self, x, evalMode=False):
        output=x
        output=self.layers(output)
#         temp =output
#         output = F.avg_pool2d(output, [output.size(2), output.size(3)], stride=1)
#         output = output.reshape(output.shape[0], output.shape[1])
        
#         label_output=self.classifier(output)
#         #print("WEIGHTS",self.classifier[0].weight.shape)
#         #label_output = label_output / torch.norm(self.classifier.weight, dim=1)
#         label_output = label_output / torch.norm(self.classifier[0].weight, dim=1)
        print("First output" , output)
        temp =output
        output = F.avg_pool2d(output, [output.size(2), output.size(3)], stride=1)
        print("Second output" , output)
        output = output.reshape(output.shape[0], output.shape[1])
        print("Third output" , output)
        label_output=self.classifier(output)
        print("Fourth output" , label_output)
        #print("WEIGHTS",self.classifier[0].weight.shape)
        label_output = label_output / torch.norm(self.classifier[0].weight, dim=1)
        print("Fifth output" , label_output)
        
 

        return None, label_output
        


def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)
        
        
# input_channels=3
# hidden_sizes=[32,16,24,32,64,96,160,320]
# repeat_list=[1,2,3,4,3,3,1]
# strides_list=[1,2,2,2,1,2,1]
# expansion_factor_list=[1,6,6,6,6,6,6]
# x=Network_MobileNet(input_channels ,hidden_sizes ,10,10 , expansion_factor_list ,strides_list , repeat_list)
# print(self.classifier[0].weight)

### Training & Testing Model

In [33]:
def train(model, data_loader,task='Classification'):
    model.train()

    

    for epoch in range(start_epoch, numEpochs):
        avg_loss = 0.0
        

#         lr = learningRate * (0.1)
        
#         for param_group in optimizer.param_groups:
#             param_group['lr'] = lr
        
        print("Entered epoch")
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(feats)[1]

            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()

            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0    
            
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
            
        torch.save({
        'epoch': epoch,
        'model_state_dict': network.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss,
        },"model_params_new" + str(epoch) + ".tar" )
            
            
            
            #print("Finished one loop")
            #break
        #print("One epoch passed")
        #break
        
#         if task == 'Classification':
#             val_loss, val_acc = test_classify(model, test_loader)
#             train_loss, train_acc = test_classify(model, data_loader)
#             print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
#                   format(train_loss, train_acc, val_loss, val_acc))
#         else:
#             test_verify(model, test_loader)


def test_classify(model, test_loader):
    print("HI")
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0
    pred_label_list=np.zeros((1))
    id1=np.zeros(1)
    i=5000
    
    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)[1]
        #print(i)
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        #print(pred_labels.flatten())
        pred_labels = pred_labels.view(-1)
        #id1=i.to(device)
        
        #print("Batch num",batch_num)
        #x=np.ndarray.flatten(id1.data.cpu().numpy())
            
        #y= np.ndarray.flatten(pred_labels.data.cpu().numpy())
        x=np.ndarray.flatten(pred_labels.data.cpu().numpy())
           
        #x_y = np.append(x,y)
        #print(x_y)
        
        pred_label_list= np.vstack((pred_label_list,x))
        id1=np.vstack((id1,i))
        
        #print(id1)
        
        
        loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels
        i+=1

    model.train()
    return np.mean(test_loss), accuracy/total , pred_label_list[1:,:],id1[1:,:]




In [34]:
with open ("test_trials_verification_student.txt", "r") as myfile:
    data=myfile.readlines()

image_pair_list=[]
for i in range(len(data)):
  
    image_pair_list.append(data[i].replace("\n","").split(" "))
    #print(image_pair_list)
    break


In [67]:
#print(len(image_pair_list))
path="/home/ubuntu/hw2part2/test_verification/"



def test_verify(model):
    #score_list=np.zeros((1))
    #file_list =np.zeros((1))
    model.eval()
    with open("output_verification2.csv",'a+') as outfile:
        for i in range(len(image_pair_list)):
            print(i)
            image1_name,image2_name = image_pair_list[i][0], image_pair_list[i][1]
            img1, img2 = Image.open(path+image1_name),Image.open(path+image2_name)
            img1, img2 = torchvision.transforms.ToTensor()(img1),torchvision.transforms.ToTensor()(img2)
            im1, im2 = torch.cuda.FloatTensor(np.array(img1).reshape((1,3,32,32))),torch.cuda.FloatTensor(np.array(img2).reshape((1,3,32,32)))
            print("Image1",im1)
            output1, output2 = model(im1)[1] , model(im2)[1]
            print("output1",output1)
            print("output2",output2)
            cos = nn.CosineSimilarity(dim=1 ,eps=1e-08)
            score = cos(output1, output2).cpu().detach()
            #norm_op1 ,norm_op2= F.softmax(output1, dim=1),F.softmax(output2, dim=1)            
            #l2 = nn.PairwiseDistance(p=2)
            #score = l2(norm_op1,norm_op2)
            print("SCore",score)
            
            filename = image1_name + " " + image2_name
            outfile.write(filename + "," + str(float(score)) + '\n')
            #break
            #score1=np.ndarray.flatten(score.data.cpu().numpy())
        #score_list=np.vstack((score_list,score1))
        
        #file_list=np.vstack((file_list,np.array([image1_name + " " + image2_name])))
        #print(file_list)
            #if(i==20):
    return score



#### Dataset, DataLoader and Constant Declarations

In [36]:
train_dataset = torchvision.datasets.ImageFolder(root='train_data/medium/', 
                                                 transform=torchvision.transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=256, 
                                               shuffle=True, num_workers=8)



# dev_dataset = torchvision.datasets.ImageFolder(root='medium_dev/', 
#                                                transform=torchvision.transforms.ToTensor())
# dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=10, 
#                                              shuffle=True, num_workers=8)

test_dataset = torchvision.datasets.ImageFolder(root='test_classification/', 
                                               transform=torchvision.transforms.ToTensor())
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, 
                                             shuffle=False, num_workers=8)


#test_verification = torchvision.datasets.ImageFolder(root='test_verification/', 
                                               #transform=torchvision.transforms.ToTensor())
#test_verify_dataloader = torch.utils.data.DataLoader(test_verification, batch_size=1, 
                                             #shuffle=True, num_workers=8)

In [72]:
# network = Network(num_feats, hidden_sizes, num_classes)
input_channels=3
hidden_sizes=[32,16,24,32,64,96,160,320]
repeat_list=[1,2,3,4,3,3,1]
strides_list=[1,1,1,1,1,2,2]
expansion_factor_list=[1,6,6,6,6,6,6]
learningRate = 3e-2
weightDecay = 5e-5
network = Network_MobileNet(input_channels ,hidden_sizes ,10,10 , expansion_factor_list ,strides_list , repeat_list)
network.apply(init_weights)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9 ,nesterov=True)


In [65]:
network.train()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network.to(device)

checkpoint = torch.load("model_params_new_11.tar")   
network.cuda()
network.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

start_epoch = epoch
numEpochs = 15
train(network, train_dataloader)



Entered epoch


KeyboardInterrupt: 

In [ ]:
# torch.save({
#             'epoch': 11,
#             'model_state_dict': network.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': 0.6588,
#             },"model_params_new_11" + ".tar" )



In [73]:
checkpoint = torch.load("model_params_new_11.tar")   
network.cuda()
network.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# lr = learningRate * (0.1 ** (epoch // 1))
# for param_group in optimizer.param_groups:
#         param_group['lr'] = lr

In [74]:
testset = ImageDataset(img_list , label_list)
test_verification_loader = DataLoader(testset , batch_size = 2048 , shuffle =False , num_workers =0 , drop_last = False)

In [75]:
def verify(model , test_loader):
    
    with torch.no_grad():
        model.eval()
        model.to(device)
        output_array = {}
        for batch_idx , (data , target) in enumerate(test_loader):
            #print(target)
            #target1=target.data.numpy()
            #print(type(target1))
   
                
            start_time = time.time()
            data = data.to(device)
            print("Shape",data.shape)
            embedding = model(data)[1]
            for i in range(target.shape[0]):
                output_array[str(target[i].item())] = embedding[i]
                
            end_time = time.time()
            print("Time ",str(end_time - start_time))
    return output_array

In [ ]:
embeddings = verify(network, test_verification_loader)

In [61]:
cos = nn.CosineSimilarity(dim=1 ,eps=1e-08)
image1 =[]
image2 = []
with open('test_trials_verification_student.txt' , 'r') as file:
    for line in file:
        temp1 = line.split()[0]
        temp2 = line.split()[1]
        image1.append(temp1.split('.')[0])
        image2.append(temp2.split('.')[0])

In [63]:
answer = []
for x,y in zip(image1, image2):
    value = cos(embeddings[x].reshape(1,-1) , embeddings[y].reshape(1,-1))
    answer.append([str(x) + '.jpg' + ' ' + str(y) + '.jpg' , value.item()])
print(answer[0])
    

['299317.jpg 248776.jpg', 0.6447991728782654]


In [66]:
file = open('v.csv' , 'w' , newline='')
with file:
    a=csv.writer(file)
    a.writerows(answer)

In [66]:
#print(embeddings.keys())
print(embeddings['299317'])

tensor([ 3.7152,  2.6177, -1.8686,  ..., -0.2592, -3.2146, -1.3829],
       device='cuda:0')


In [69]:
#network.load_state_dict(torch.load("model_params_new.tar"))

score= test_verify(network)
# print(file_list)
# print(score_list)
# output = np.hstack((file_list,score_list))
# np.savetxt("output_verification_1.csv", output, delimiter=',')

0
Image1 tensor([[[[0.5725, 0.4118, 0.4431,  ..., 0.6980, 0.6902, 0.7020],
          [0.6157, 0.4392, 0.4431,  ..., 0.6667, 0.6941, 0.7098],
          [0.6745, 0.4941, 0.4863,  ..., 0.6588, 0.6745, 0.6745],
          ...,
          [0.2353, 0.1412, 0.2275,  ..., 0.4275, 0.4863, 0.4039],
          [0.4235, 0.3569, 0.3490,  ..., 0.5412, 0.5216, 0.4275],
          [0.4078, 0.3804, 0.3490,  ..., 0.5843, 0.4863, 0.4980]],

         [[0.4275, 0.2667, 0.2980,  ..., 0.5059, 0.4980, 0.5098],
          [0.4706, 0.2941, 0.2980,  ..., 0.4745, 0.5020, 0.5176],
          [0.5216, 0.3412, 0.3333,  ..., 0.4667, 0.4824, 0.4824],
          ...,
          [0.1608, 0.0549, 0.1255,  ..., 0.2941, 0.3725, 0.3020],
          [0.3569, 0.2706, 0.2353,  ..., 0.3922, 0.4000, 0.3137],
          [0.3373, 0.2902, 0.2235,  ..., 0.4235, 0.3529, 0.3765]],

         [[0.5882, 0.4275, 0.4588,  ..., 0.7255, 0.7176, 0.7294],
          [0.6314, 0.4549, 0.4588,  ..., 0.6941, 0.7216, 0.7373],
          [0.6941, 0.5137, 0.5059

In [13]:
# checkpoint = torch.load("model_params_new.tar")
# network.cuda()
# network.load_state_dict(checkpoint['model_state_dict'])
# epoch = checkpoint['epoch']
# network.eval()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mean,accuracy,pred_list,id1 = test_classify(network, test_dataloader)
#print(pred_list.shape)
#print(id1.shape)
#print(pred_list)


HI


In [14]:
#print(pred_list[0])
#print(pred_list)
dictionary1=train_dataset.classes
pred_list1 = np.zeros(1)
for i in range(pred_list.shape[0]):
    #pred_list1=np.vstack((pred_list1,dictionary[str(int(pred_list[i][0]))]))
    #print(pred_list[i][0])
    #print(dictionary1[int(pred_list[i][0])])
    pred_list1=np.vstack((pred_list1, float(dictionary1[int(pred_list[i][0])])))

output=np.hstack((id1,pred_list1[1:]))
#print(output.shape)
print(output[0])

[5000.  947.]


In [15]:

np.savetxt("output_classification_7.csv", output, delimiter=',')